In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run_test

import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [1]:
from helper import double_sha256

bit_field_size = 10
bit_field = [0] * bit_field_size

h = double_sha256(b'hello world')
bit = int.from_bytes(h, 'big') % bit_field_size
bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [2]:
from helper import double_sha256

bit_field_size = 10
bit_field = [0] * bit_field_size

for item in (b'hello world', b'goodbye'):
    h = double_sha256(item)
    bit = int.from_bytes(h, 'big') % bit_field_size
    bit_field[bit] = 1
print(bit_field)

[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


### Exercise 1

#### Calculate the Bloom Filter for 'hello world' and 'goodbye' using the `hash160` hash function over a bit field of 10.

In [ ]:
# Exercise 1

from helper import hash160

bit_field_size = 10
bit_field = [0] * bit_field_size

In [3]:
from helper import double_sha256, hash160

bit_field_size = 10
bit_field = [0] * bit_field_size

for item in (b'hello world', b'goodbye'):
    for hash_function in (double_sha256, hash160):
        h = hash_function(item)
        bit = int.from_bytes(h, 'big') % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [4]:
from helper import murmur3
from bloomfilter import BIP37_CONSTANT

field_size = 2
num_functions = 2
tweak = 42

bit_field_size = field_size * 8
bit_field = [0] * bit_field_size

for phrase in (b'hello world', b'goodbye'):
    for i in range(num_functions):
        seed = i * BIP37_CONSTANT + tweak
        h = murmur3(phrase, seed=seed)
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0]


### Exercise 2

#### Given a Bloom Filter with these parameters: size=10, function count=5, tweak=99, which bits are set after adding these items? 

 * `b'Hello World'`
 * `b'Goodbye!'`


In [ ]:
# Exercise 2

from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')

### Exercise 3

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass.

In [ ]:
# Exercise 3

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_add'))

### Exercise 4

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass.

In [ ]:
# Exercise 4

reload(bloomfilter)
run_test(bloomfilter.BloomFilterTest('test_filterload'))

### Exercise 5

#### Make [this test](/edit/code-ch12/network.py) pass.

In [ ]:
# Exercise 5

reload(network)
run_test(network.GetDataMessageTest('test_serialize'))

In [5]:
from bloomfilter import BloomFilter
from helper import decode_base58
from merkleblock import MerkleBlock
from network import (
    FILTERED_BLOCK_DATA_TYPE,
    GetHeadersMessage, GetDataMessage, HeadersMessage,
    SimpleNode, 
)
from tx import Tx

last_block_hex = '00000000000538d5c2246336644f9a4956551afb44ba47278759ec55ea912e19'
address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
h160 = decode_base58(address)
node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=True)
bf = BloomFilter(30, 5, 90210)
bf.add(h160)
node.handshake()
node.send(b'filterload', bf.filterload())
start_block = bytes.fromhex(last_block_hex)
getheaders_message = GetHeadersMessage(start_block=start_block)
node.send(b'getheaders', getheaders_message.serialize())
headers_envelope = node.wait_for_commands({b'headers'})
stream = headers_envelope.stream()
headers = HeadersMessage.parse(stream)
get_data_message = GetDataMessage()
for b in headers.blocks:
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    get_data_message.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())
node.send(b'getdata', get_data_message.serialize())
while True:
    envelope = node.wait_for_commands({b'merkleblock', b'tx'})
    stream = envelope.stream()
    if envelope.command == b'merkleblock':
        mb = MerkleBlock.parse(stream)
        if not mb.is_valid():
            raise RuntimeError('invalid merkle proof')
    else:
        prev_tx_obj = Tx.parse(stream, testnet=True)
        for i, tx_out in enumerate(prev_tx_obj.tx_outs):
            if tx_out.script_pubkey.address(testnet=True) == address:
                print('found: {}:{}'.format(prev_tx_obj.hash().hex(), i))

NotImplementedError: 

### Exercise 6

#### Get the current testnet block id, send yourself some testnet coins, find the UTXO corresponding to the testnet coin *without using a block explorer*, create a transaction using that UTXO as an input and broadcast that message on the network.

In [ ]:
# Exercise 6

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import double_sha256, little_endian_to_int, encode_varint, read_varint, decode_base58, p2pkh_script, SIGHASH_ALL
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from tx import Tx, TxIn, TxOut